In [1]:
import pandas as pd

#load filtered jump point data
df = pd.read_csv("../data/filtered_jump_points.csv")

#show first few rows
df.head()

,patient_id,label,timestamp,event_type,event_value
0,1,0,2025-01-01 08:00:10.200,flow,22.988039
1,1,0,2025-01-01 08:00:33.800,flow,26.679169
2,1,0,2025-01-01 08:00:34.600,flow,8.291424
3,1,0,2025-01-01 08:00:35.400,flow,12.047348
4,1,0,2025-01-01 08:00:35.600,flow,4.242724


In [4]:
#each jump point row becomes a triple: (event_type, event_value, time_gap)
#covert timestamp column to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])

#sort by timestamp(safety - should be ordered already)
df = df.sort_values("timestamp").reset_index(drop=True)

#calculate time gap (in seconds) between events
df["time_gap"] = df["timestamp"].diff().dt.total_seconds().fillna(0)

#show first 10 results
df.head(10)

,patient_id,label,timestamp,event_type,event_value,time_gap
0,1,0,2025-01-01 08:00:02.200,pressure,12.871221,0.0
1,1,0,2025-01-01 08:00:02.400,pressure,3.561661,0.2
2,1,0,2025-01-01 08:00:02.800,pressure,7.087086,0.4
3,1,0,2025-01-01 08:00:06.800,pressure,9.052294,4.0
4,1,0,2025-01-01 08:00:07.000,pressure,4.497117,0.2
5,1,0,2025-01-01 08:00:07.400,pressure,9.563705,0.4
6,1,0,2025-01-01 08:00:07.600,pressure,16.166415,0.2
7,1,0,2025-01-01 08:00:08.400,pressure,8.416227,0.8
8,1,0,2025-01-01 08:00:09.200,pressure,13.744280,0.8
9,1,0,2025-01-01 08:00:09.400,pressure,5.945877,0.2
